In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

In [3]:
weather_data = pd.read_csv("weather_data_cities.csv")
weather_data

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Gīdam,19,IN,1602134856,64,18.98,81.40,88.65,2.04
1,1,Arrondissement de Mayenne,90,FR,1602134856,93,48.33,-0.67,59.00,9.17
2,2,Souillac,40,MU,1602134512,74,-20.52,57.52,78.80,18.34
3,3,Acaraú,69,BR,1602134856,88,-2.89,-40.12,75.96,11.45
4,4,Ushuaia,75,AR,1602134423,89,-54.80,-68.30,39.20,4.85
...,...,...,...,...,...,...,...,...,...,...
582,582,Ziro,40,IN,1602134764,83,27.63,93.83,84.20,3.36
583,583,Bendigo,100,AU,1602134973,75,-36.77,144.28,55.99,5.99
584,584,Sijunjung,100,ID,1602134973,77,-0.69,100.95,78.67,2.80
585,585,Atambua,98,ID,1602134780,59,-9.11,124.89,84.85,2.01


In [5]:
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations
locations = weather_data[["Lat", "Lng"]]

# Use Humidity as the weight
humidity = weather_data["Humidity"]

In [8]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=50, point_radius=1)

# Add heat layer for the display
fig.add_layer(heat_layer)

# Display the Humidity Hit Map now

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & \
                                        (weather_data["Cloudiness"] == 0) & \
                                       (weather_data["Max Temp"] >= 70) & \
                                       (weather_data["Max Temp"] <= 80)].dropna()
city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
65,65,Graaff-Reinet,0,ZA,1602134870,48,-32.25,24.53,70.39,3.94
70,70,Maryborough,0,AU,1602134871,45,-25.53,152.70,78.21,9.01
174,174,Joshīmath,0,IN,1602134891,21,30.57,79.57,71.37,6.33
226,226,Erzin,0,TR,1602134545,72,36.96,36.20,71.60,1.12
234,234,Pahrump,0,US,1602134455,16,36.21,-115.98,70.00,1.01
275,275,Floresta,0,BR,1602134766,79,-8.60,-38.57,71.55,6.64
315,315,Beroroha,0,MG,1602134918,53,-21.67,45.17,78.12,3.49
330,330,Sakakah,0,SA,1602134921,17,29.97,40.21,73.40,3.13
380,380,Chaman,0,PK,1602134931,19,30.92,66.45,73.15,4.18
422,422,Jalu,0,LY,1602134770,24,29.03,21.55,76.24,6.96


In [16]:
hotels = city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotels["Hotel Name"] = ""
hotels

,City,Country,Lat,Lng,Hotel Name
65,Graaff-Reinet,ZA,-32.25,24.53,
70,Maryborough,AU,-25.53,152.70,
174,Joshīmath,IN,30.57,79.57,
226,Erzin,TR,36.96,36.20,
234,Pahrump,US,36.21,-115.98,
275,Floresta,BR,-8.60,-38.57,
315,Beroroha,MG,-21.67,45.17,
330,Sakakah,SA,29.97,40.21,
380,Chaman,PK,30.92,66.45,
422,Jalu,LY,29.03,21.55,


In [18]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#parameters to search for hotels with 5000 meters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [22]:
for index, row in hotels.iterrows():
    city_name = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotels.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result ... skipping ")      
    print("--------------------------------------------")
    time.sleep(1)
print("--------------------------End of Hotel Search--------------------------")

Retrieving Results for Index 65: Graaff-Reinet.
Closest hotel in Graaff-Reinet is Buiten Verwagten.
--------------------------------------------
Retrieving Results for Index 70: Maryborough.
Closest hotel in Maryborough is Carriers Arms Hotel Motel.
--------------------------------------------
Retrieving Results for Index 174: Joshīmath.
Closest hotel in Joshīmath is Shivalik Camps & Resort.
--------------------------------------------
Retrieving Results for Index 226: Erzin.
Closest hotel in Erzin is Hattusa Vacation Thermal Club Erzin.
--------------------------------------------
Retrieving Results for Index 234: Pahrump.
Closest hotel in Pahrump is Pahrump Nugget.
--------------------------------------------
Retrieving Results for Index 275: Floresta.
Closest hotel in Floresta is Hotel Parque Das Caraibeiras.
--------------------------------------------
Retrieving Results for Index 315: Beroroha.
Missing field/result ... skipping 
--------------------------------------------
Retriev

In [24]:
hotels

,City,Country,Lat,Lng,Hotel Name
65,Graaff-Reinet,ZA,-32.25,24.53,Buiten Verwagten
70,Maryborough,AU,-25.53,152.70,Carriers Arms Hotel Motel
174,Joshīmath,IN,30.57,79.57,Shivalik Camps & Resort
226,Erzin,TR,36.96,36.20,Hattusa Vacation Thermal Club Erzin
234,Pahrump,US,36.21,-115.98,Pahrump Nugget
275,Floresta,BR,-8.60,-38.57,Hotel Parque Das Caraibeiras
315,Beroroha,MG,-21.67,45.17,
330,Sakakah,SA,29.97,40.21,Raoum Inn Hotel
380,Chaman,PK,30.92,66.45,Sakhy Hotal Wah Mosafir Khana
422,Jalu,LY,29.03,21.55,Jalu Hotel


In [26]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotels_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
locations = hotels[["Lat", "Lng"]]

In [28]:
markers = gmaps.marker_layer(locations, info_box_content = hotels_info)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))